In [ ]:
####################################
#ENVIRONMENT SETUP

In [ ]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [ ]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [ ]:
def GetPlottingDirectory(plotFileName, plotType):
    plottingDirectory = mainCodeDirectory=os.path.join(mainDirectory,"Code","PLOTTING")
    
    specificPlottingDirectory = os.path.join(plottingDirectory, plotType, 
                                             f"{ModelData.res}_{ModelData.t_res}_{ModelData.Nz_str}nz")
    os.makedirs(specificPlottingDirectory, exist_ok=True)

    plottingFileName=os.path.join(specificPlottingDirectory, plotFileName)

    return plottingFileName

def SaveFigure(fig,plotType, fileName):
    plotFileName = f"{fileName}_{ModelData.res}_{ModelData.t_res}_{ModelData.Np_str}.jpg"
    plottingFileName = GetPlottingDirectory(plotFileName, plotType)
    print(f"Saving figure to {plottingFileName}")
    fig.savefig(plottingFileName, dpi=300, bbox_inches='tight')

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [ ]:
#IMPORT CLASSES
sys.path.append(os.path.join(mainCodeDirectory,"3_Project_Algorithms","1_Domain_Profiles"))
from CLASSES_DomainProfiles import DomainProfiles_Class, DomainProfiles_DataLoading_Class

In [ ]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
dataName="Budgets"

#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=2)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="Domain_Profiles", dataName=dataName,
                                dtype='float32',codeSection = "Project_Algorithms")

In [ ]:
####################################
#PLOTTING FUNCTIONS

In [ ]:
def LoadMeanLFC():
    # dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    # in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    # with open(in_file, 'rb') as f:
    #     MeanLFC = pickle.load(f)
    MeanLFC=2 #*#*
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC}\n")


def LoadAllCloudBase():
    # dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    # in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    # with open(in_file, 'rb') as f:
    #     all_cloudbase = pickle.load(f)
    all_cloudbase=1 #*#*
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {cloudbase}\n")

In [ ]:
#thresholds
w_thresh1,w_thresh2,qcqi_thresh = DomainProfiles_DataLoading_Class.GetCloudyUpdraftThresholds()

In [ ]:
def LoadProfiles(dataName, datatype, masked, varNames):
    Dictionary = DomainProfiles_DataLoading_Class.LoadProfile(ModelData,DataManager, dataName=dataName, datatype=datatype,timeString = "timeaverage",masked=masked)

    Dictionary_SE = {}
    for varName in varNames:
        one=Dictionary[f"{varName}_{datatype}_timeaverage"]
        two=Dictionary[f"{varName}_squares_{datatype}_timeaverage"]
        Dictionary_SE[f"{varName}_{datatype}_timeaverage_SE"] = ProfileStandardError(one,two)

    factor=1.96
    return Dictionary, Dictionary_SE, factor

In [ ]:
def GetMultiplyFactor(varName):
    if "qvb" in varName:
        multiplyFactor = 1000

    else:
        multiplyFactor = 1
    return multiplyFactor

def PlotProfile(profile, axis, label, color, xlabel, linestyle='solid'):    
    axis.plot(profile[:, 0], profile[:, 1], label=label, color=color, linestyle=linestyle)

    axis.set_xlabel(xlabel)
    axis.set_ylabel("z (km)")
    axis.grid(True)
    
def PlotProfile_SE(profileMean,profile_SE,color, factor, axis):
    axis.fill_betweenx(profileMean[:, 1],
                       profileMean[:, 0] - factor*profile_SE[:, 0],
                       profileMean[:, 0] + factor*profile_SE[:, 0],
                       color=color, alpha=0.1)

def PlotBudgets(axis, dataName, varNames, maskeds, linestyles, datatypes, xlabel):
    """
    Plot vertical profiles for each budget variable in varNames on chosen axis.
    """

    # consistent color scheme for datatypes
    colors = ['grey', 'orange', 'green', 'red',
              'pink', 'brown', 'purple', 'blue']

    for (datatype, masked, linestyle) in zip(datatypes, maskeds, linestyles):

        Dictionary, Dictionary_SE, factor = LoadProfiles(dataName, datatype, masked, varNames)
            
        for count, varName in enumerate(varNames):
            
            profile = Dictionary[f"{varName}_{datatype}_timeaverage"]
            profileMean = ProfileMean(profile)

            profile_SE = Dictionary_SE[f"{varName}_{datatype}_timeaverage_SE"]

            
            if datatype == 'all':
                label = f"domain"

            if datatype in ['cloudy']:
                label = f"{varName}"
            else:
                label = None    

            multiplyFactor = GetMultiplyFactor(varName)
            profileMean[:,0]*=multiplyFactor
            profile_SE[:,0]*=multiplyFactor
            
            PlotProfile(profileMean, axis, label=label, color=colors[count], xlabel=xlabel, linestyle=linestyle)
            PlotProfile_SE(profileMean,profile_SE, colors[count], factor, axis)

        axis.set_ylim(0,20)
        axis.grid(True)

    axis.legend()


    #FIXING TICKS
    axs = fig.get_axes()
    SnapLimitsToTicks(axs, dim='x')

    apply_scientific_notation([axis])

    #ADDING CLOUDBASE AND LFC
    axline_lw = 1.2
    axis.axhline(cloudbase, color='purple', linestyle='dashed', lw=axline_lw, zorder=-100)
    axis.axhline(MeanLFC, color='green', linestyle='dashed', lw=axline_lw, zorder=-100)
    axis.axvline(0,color='gray',linestyle='dashed',zorder=-10)

    #FIXING LEGEND *#*
    axis.legend(fontsize='small')

In [ ]:
####################################
#RUNNING

In [ ]:
fig = plt.figure(figsize=(15, 7))
fig.subplots_adjust(top=0.95, wspace=0.2, hspace=0.2)
gs = fig.add_gridspec(1, 3)

datatypes = ['all','general','cloudy']
maskeds = [False,True,True]
linestyles = ['dotted','dashed','solid']

#####################
# #####################

dataName = "W_Budgets"
varNames = [
        'wb_hadv', 'wb_vadv', 'wb_hidiff', 'wb_vidiff',
        'wb_hturb', 'wb_vturb', 
        'wb_pgrad', 'wb_buoy'
    ]
xlabel = "W Budgets " + r"$(m/s^2)$"
axis=fig.add_subplot(gs[0, 0])
PlotBudgets(axis, dataName, varNames, maskeds, linestyles, datatypes, xlabel)

#####################
#####################

dataName = "QV_Budgets"
varNames = [
    'qvb_hadv', 'qvb_vadv', 'qvb_hidiff', 'qvb_vidiff',
    'qvb_hturb', 'qvb_vturb', 
    'qvb_mp'
]
xlabel="QV Budgets " + "$(g/kg/s)$"
axis=fig.add_subplot(gs[0, 1])
PlotBudgets(axis, dataName, varNames, maskeds, linestyles, datatypes, xlabel)


#####################
#####################

dataName = "TH_Budgets"
varNames = [
    'ptb_hadv', 'ptb_vadv', 'ptb_hidiff', 'ptb_vidiff',
    'ptb_hturb', 'ptb_vturb', 
    'ptb_mp',# 'ptb_rad',
    #'ptb_div', 'ptb_diss'
]
xlabel="TH Budgets " + r"$(K/s)$"
axis=fig.add_subplot(gs[0, 2])
PlotBudgets(axis, dataName, varNames, maskeds, linestyles, datatypes, xlabel)

#####################
#####################

title = "Updraft BUDGET General (dashed) and Cloudy (solid) Updraft Domain Profiles "
fig.suptitle(title)

fileName=f"Eulerian_Profiles_Budgets" 
SaveFigure(fig,plotType="Project_Algorithms/Domain_Profiles/Eulerian_Profiles_Budgets",fileName=fileName)